In [2]:
#Imports for M-LOOP
# import mloop.interfaces as mli
# import mloop.controllers as mlc
# import mloop.visualizations as mlv

#Other imports
import numpy as np
import torch
import matplotlib.pyplot as plt
from tqdm.notebook import trange
# import time

from functions import*
from Mloop_and_simcim import*

datatype = torch.float32
device = 'cuda'

In [ ]:
# def energy(J,b,s):
#     return -0.5*torch.einsum('in,ij,jn->n',s,J,s) - torch.einsum('in,ik->n',s,b)

# def generate_cities(N_cities):
#     cities = np.random.randn(N_cities,2)
#     L = (cities.reshape(cities.shape[0],1,2) - cities.reshape(1,cities.shape[0],2))**2
#     lengths = np.sqrt(np.sum(L,2))
#     return cities,lengths

# def plot_cities(ax,cities,order):
#     N_cities = cities.shape[0]
#     if order.shape[0]==N_cities:
#         cities_rearranged = cities[order,:]
#         for i in range(N_cities-1):
#             ax.plot(cities_rearranged[i:i+2,0],cities_rearranged[i:i+2,1],'-ro')
# #             x,y = (cities_rearranged[i,0],cities_rearranged[i,1])
# #             ax.annotate(str(order[i]),(x,y),xytext=(x+0.01*x,y+0.01*y),fontsize=12)
#         ax.plot([cities_rearranged[N_cities-1,0],cities_rearranged[0,0]],
#             [cities_rearranged[N_cities-1,1],cities_rearranged[0,1]],'-ro')
# #         x,y = (cities_rearranged[-1,0],cities_rearranged[-1,1])
# #         ax.annotate(str(order[-1]),(x,y),xytext=(x+0.01*x,y+0.01*y),fontsize=12)
#         ax.set_xlabel('x',fontsize=18)
#         ax.set_ylabel('y',fontsize=18)
#         ax.tick_params(labelsize=18)
#         ax.grid()
#     else:
#         ax.text(0.5, 0.5,'Решения не найдено',
#          horizontalalignment='center',
#          verticalalignment='center',
#          transform = ax.transAxes)
    
# def length(order,lengths):
#     order1 = torch.zeros(order.shape[0],dtype=torch.long)
#     order1[:-1] = order[1:]
#     order1[-1] = order[0]
#     return torch.sum(lengths[(order,order1)])   

# def Qubo(lengths,A,B):
#     N_cities = lengths.shape[0]
#     Q = np.zeros((N_cities,N_cities,N_cities,N_cities))
#     inds0 = np.arange(N_cities)
#     inds1 = np.concatenate((inds0[1:],inds0[0:1]))
#     Q[:,inds0,:,inds1] += B*np.repeat(lengths.reshape(1,N_cities,N_cities),N_cities,axis=0)
#     dims = np.arange(N_cities)
#     Q[dims,:,dims,:] += A
#     inds0 = dims.reshape(N_cities,1).repeat(N_cities,axis=1).flatten()
#     inds1 = dims.reshape(1,N_cities).repeat(N_cities,axis=0).flatten()
#     Q[inds0,inds1,inds0,inds1] -= Q[inds0,inds1,inds0,inds1]
#     Q[:,dims,:,dims] += A
#     Q[inds1,inds0,inds1,inds0] -= Q[inds1,inds0,inds1,inds0]
#     b = -np.ones((N_cities,N_cities))*2*A
#     return Q,b

# def get_Jh(lengths,A,B):
#     Q,b = Qubo(lengths,A,B)
#     Q = torch.tensor(Q.reshape(N_cities**2,N_cities**2),dtype = torch.float32)
#     b = torch.tensor(b.reshape(N_cities**2),dtype = torch.float32)
#     Q = 0.5*(Q + Q.t())
#     J = -0.5*Q
#     h = -0.5*(Q.sum(1) + b)
#     h = h.reshape(-1,1)
#     return J,h

# def H(Q,b,x):
#     return torch.einsum('ij,ni,nj->n',Q,x,x) + x@b 

# def int2base(nums,base,N_digits):
#     nums_cur = torch.clone(nums)
#     res = torch.empty((nums.shape[0],N_digits))
#     for i in range(N_digits):
#         res[:,N_digits-1-i] = torch.remainder(nums_cur,base)
#         nums_cur = (nums_cur/base).type(torch.long)
#     return res

# def get_order(x,lengths,A,B):
#     Q,b = Qubo(lengths,A,B)
#     Q = torch.tensor(Q.reshape(N_cities**2,N_cities**2),dtype = torch.float32)
#     b = torch.tensor(b.reshape(N_cities**2),dtype = torch.float32)
#     Q = 0.5*(Q + Q.t())
#     ind_min = torch.argmin(H(Q,b,x.type(torch.float32)))
#     inds_nonzero = np.nonzero(x[ind_min].reshape(N_cities,N_cities))
#     inds_order = (inds_nonzero[:,1].sort()[1])
#     order = inds_nonzero[:,0][inds_order]
#     return order

# def get_order_simcim(s_min,N_cities):
#     inds_nonzero = np.nonzero((0.5*(s_min+1)).reshape(N_cities,N_cities))
#     inds_order = (inds_nonzero[:,1].sort()[1])
#     order = inds_nonzero[:,0][inds_order]
#     return order

In [ ]:
# class Simcim(mli.Interface):
    
#     params_disc = {}
#     params_cont = {}

#     params_cont['c_th'] = 1.
#     params_cont['zeta'] = 1.

#     params_disc['N'] = 2000
#     params_disc['attempt_num'] = 1000
#     params_cont['dt'] = 100.
#     params_cont['sigma'] = 1
#     params_cont['alpha'] = 0.9

#     params_cont['S'] = torch.tensor(1.)
#     params_cont['D'] = torch.tensor(-0.2)
#     params_cont['O'] = torch.tensor(0.2)  
        
#     def __init__(self,J,b,device,datatype):
#         super(Simcim,self).__init__()

#         self.N = self.params_disc['N']
#         self.attempt_num = self.params_disc['attempt_num']
#         self.J = J.type(datatype).to(device)
#         self.b = b.type(datatype).to(device)
#         self.Jmax = torch.max(torch.sum(torch.abs(J),1))
#         self.dim = J.shape[0]
        
#         self.device = device
#         self.datatype = datatype
        
    
#     # amplitude increment
#     def ampl_inc(self,c,p):
#         return ((p*c + self.zeta*(torch.mm(self.J,c)+self.b))*self.dt
#             + (self.sigma*torch.randn((c.size(0),self.attempt_num),dtype=self.datatype).to(self.device)))
    
#     # pump ramp
#     def pump(self):
#         i = torch.arange(self.N,dtype=self.datatype).to(self.device)
#         arg = torch.tensor(self.S,dtype = self.datatype).to(self.device)*(i/self.N-0.5)
#         return self.Jmax*self.O*(torch.tanh(arg) + self.D )

#     def pump_lin(self):
#         t = self.dt*torch.arange(self.N,dtype=self.datatype).to(self.device)
#         eigs = torch.eig(self.J)[0][:,0]
#         eig_min = torch.min(eigs)
#         eig_max = torch.max(eigs)
#         p = -self.zeta*eig_max + self.zeta*(eig_max-eig_min)/t[-1]*t
#         return p
    
#     # amplitude initializer 
#     def init_ampl(self):
#         return torch.zeros((self.dim, self.attempt_num),dtype=self.datatype).to(self.device)
    
#     def bias_sch(self):
#         return torch.linspace(0,1,self.N,dtype=self.datatype).to(self.device)
        
    
#     def tanh(self,c):
#         return self.c_th*torch.tanh(c)
    
#     # evolution of amplitudes
#     # N -- number of time iterations
#     # attempt_num -- number of runs 
#     # J -- coupling matrix
#     # b -- biases
#     # O, S, D -- pump parameters
#     # sigma -- sqrt(std) for random number
#     # alpha -- momentum parameter
#     # c_th -- restriction on amplitudes growth
#     def evolve(self,params_opt):
        
#         for key in params_opt.keys():
#             self.params_cont[key] = params_opt[key]
        
#         self.dt = self.params_cont['dt']
#         self.zeta = self.params_cont['zeta']
#         self.c_th = self.params_cont['c_th']
#         self.O = self.params_cont['O']
#         self.D = self.params_cont['D']
#         self.S = self.params_cont['S']
#         self.sigma = self.params_cont['sigma']
#         self.alpha = self.params_cont['alpha']
#         self.Jmax = torch.max(torch.sum(torch.abs(self.J),1))
#         self.dim = self.J.shape[0]
        
    
#         # choosing an attempt for the amplitude evolution 
#         random_attempt = np.random.randint(self.attempt_num)
    
#         # initializing current amplitudes
#         c_current = self.init_ampl()
    
#         # initializing full array of amplitudes
#     #     c_full = torch.zeros(N,dim,attempt_num)
#     #     c_full[0] = c_current
    
#         # creating the array for evolving amplitudes from random attempt
#         c_evol = torch.empty((self.dim, self.N),dtype=datatype).to(device)
#         c_evol[:,0] = c_current[:,random_attempt]
    
#         # define pump array
#         p = self.pump()
# #         p = self.pump_lin()
#         b_ramp = self.bias_sch() 
    
#         # define copupling growth
#     #     zeta = coupling(init_value,final_value,dt,N)
    
#         # initializing moving average of amplitudes increment
#         dc_momentum = torch.zeros((self.dim, self.attempt_num),dtype=self.datatype).to(self.device)
#         #free_energy_ar = torch.empty(self.N-1, dtype = self.datatype).to(device)
#         for i in trange(1,self.N):
        
#             # calculating amplitude increment
#             dc = self.ampl_inc(c_current,p[i])
#             dc /= torch.sqrt((dc**2).sum(0)).reshape(1,self.attempt_num)
        
#             # calculating moving average of amplitudes increment
#             dc_momentum = self.alpha*dc_momentum + (1-self.alpha)*dc
        
#             # calculating possible values of amplitudes on the next step
#             c1 = c_current + dc_momentum
        
#             # comparing c1 with c_th
#             th_test = (torch.abs(c1)<self.c_th).type(self.datatype)
        
#             # updating c_current
#     #         c_current = c_current + th_test*dc_momentum
#             c_current = th_test*(c_current + dc_momentum) + (1.-th_test)*torch.sign(c_current)*self.c_th
#     #         c_current = step(c_current + dc_momentum,c_th,device, datatype)
#     #         c_current = tanh(c1,c_th)
        
#             #updating c_full
#     #         c_full[i] = torch.tanh(c_full[i-1] + dc_momentum)
        
#             # add amplitude values from random attempt to c_evol array 
#             c_evol[:,i] = c_current[:,random_attempt]
            
#             #s = torch.sign(c_current)
#             #free_energy_ar[i-1] = self.free_energy(s,self.beta)
        
#         return c_current, c_evol#, free_energy_ar
    
#     def energy_cost(self,params_opt):
#         c_current, c_evol = self.evolve(params_opt)
#         s = torch.sign(c_current)
#         return -0.5*torch.einsum('in,ij,jn->n',s,self.J,s) - torch.einsum('in,ik->n',s,self.b)
    
#     def energy(self,s):
#         return -0.5*torch.einsum('in,ij,jn->n',s,self.J,s) - torch.einsum('in,ik->n',s,self.b)
    
    
#     def get_next_cost_dict(self,params_dict):        
#         #Get parameters from the provided dictionary
#         params = params_dict['params']
#         params_opt = {}
        
#         params_opt['zeta'] = params[0] 
#         params_opt['dt'] = params[1]
#         params_opt['O'] = params[2]
#         params_opt['S'] = params[3]
#         params_opt['D'] = params[4]
        
        
#         cost = torch.mean(self.energy_cost(params_opt)).cpu().numpy()
#         #There is no uncertainty in our result
#         uncer = 0
#         #The evaluation will always be a success
#         bad = False
#         #Add a small time delay to mimic a real experiment
        
#         #The cost, uncertainty and bad boolean must all be returned as a dictionary
#         #You can include other variables you want to record as well if you want
#         cost_dict = {'cost':cost, 'uncer':uncer, 'bad':bad}
#         return cost_dict
    
# def main(controller_type,interface):
# #     interface = Simcim(J,h,device,datatype)
    
#     controller = mlc.create_controller(interface,
#                                        controller_type = controller_type,
#                                        max_num_runs_without_better_params = 50,
#                                        max_num_runs = 300,
#                                        num_params = 5, 
#                                        min_boundary = [0,0,0,0,-2],
#                                        max_boundary = [10,100,5,10,2])
    
#     controller.optimize()
    
    
#     print('Best parameters found:')
#     print(controller.best_params)
    
# #     mlv.show_all_default_visualizations(controller)
#     params = controller.best_params
#     params_opt = {}
#     params_opt['zeta'] = params[0]
#     params_opt['dt'] = params[1] 
#     params_opt['O'] = params[2]
#     params_opt['S'] = params[3]
#     params_opt['D'] = params[4]
#     return params_opt
    

In [ ]:
N_cities = 20
generate_cities(N_cities)
B = 0.1
A = 1.5*B*lengths_ar[n_map].max()
J,h = get_Jh(lengths_ar[n_map], A, B)
J = J.to(device)
h = h.to(device)
simcim = Simcim(J, h, device, datatype)
c_current, c_evol = simcim.evolve(simcim.params_cont)
c_current.shape

In [ ]:
for i in range(c_evol.shape[0]):
    plt.plot(c_evol[i].cpu().numpy())
plt.show()
s_cur = torch.sign(c_current)
E = energy(J,h,s_cur)
plt.hist(E.cpu().numpy())
plt.show()

In [ ]:
lengths_sa[n_map]

In [ ]:
s_min = s_cur[:,torch.argmin(E)]
order = get_order_simcim(s_min,N_cities)

fig,ax = plt.subplots(figsize = (8,8))
plot_cities(ax,cities[n_map],order.cpu().numpy())
ax.plot(np.zeros(5),'o',color='white',alpha=0.,label =# 'Длина маршрута '+
             str(np.around(length(order,torch.tensor(lengths_ar)[n_map]).numpy(),2)))#+'\n алгоритм без нормировки')
ax.legend(fontsize = 18,loc=1)
ax.tick_params(labelsize=18)

In [ ]:
order.sort()

In [ ]:
N_cities = 40
cities,lengths = generate_cities(N_cities)
B = 0.1
A = 1.5*B*lengths.max()
J,h = get_Jh(lengths, A, B)
J = J.to(device)
h = h.to(device)
fig, ax = plt.subplots(figsize=(8,5))
plot_cities(ax,cities,np.random.permutation(cities.shape[0]))
fig.tight_layout()
plt.show()

In [ ]:
N_maps = 40
N_cities = 40
cities = np.empty((N_maps,N_cities,2))
lengths = np.empty((N_maps,N_cities,N_cities))
for i in range(N_maps):
    cities_cur,lengths_cur = generate_cities(N_cities)
#     B = 0.1
#     A = 1.5*B*lengths.max()
#     J_cur,h_cur = get_Jh(lengths, A, B)
    cities[i] = cities_cur
    lengths[i] = lengths_cur
np.save('cities_'+str(N_cities)+'cities_'+str(N_maps)+'maps',cities)  
np.save('lengths_'+str(N_cities)+'cities_'+str(N_maps)+'maps',lengths)

In [ ]:
N_maps = 40
N_cities = 40
# keys = ['gaussian_process', 'neural_net', 'differential_evolution', 'nelder_mead', 'random']
keys = ['neural_net']
params_opt = {}
lengths = np.load('lengths_40cities_40maps.npy')
for controller_type in keys:
    params_opt[controller_type] = []
    for i in range(N_maps):
        B = 0.1
        A = 1.5*B*lengths[i].max()
        J,h = get_Jh(lengths[i], A, B)
        J = J.to(device)
        h = h.to(device)
        simcim = Simcim(J, h, device, datatype)
        params_opt[controller_type].append(main(controller_type,simcim))
np.save('params_opt_nn'+str(N_cities)+'cities_'+str(N_maps)+'maps',params_opt)

In [ ]:
np.save('params_opt_2'+str(N_cities)+'cities_'+str(N_maps)+'maps',params_opt)


In [ ]:
for key in params_opt.keys():
    print(len(params_opt[key]))

In [ ]:
# params_opt = {}
# params_opt_cur = np.load('params_opt_40cities_40maps.npy',allow_pickle=True).all()
# params_opt['gaussian_process'] = params_opt_cur['gaussian_process']
# params_opt_cur = np.load('params_opt_nn40cities_40maps.npy',allow_pickle=True).all()
# params_opt['neural_net'] = params_opt_cur['neural_net']
# params_opt_cur = np.load('params_opt_de_nm_r40cities_40maps.npy',allow_pickle=True).all()
# for key in params_opt_cur.keys():
#     params_opt[key] = params_opt_cur[key]

# np.save('params_opt_'+str(N_cities)+'cities_'+str(N_maps)+'maps',params_opt)

In [ ]:
cities40 = np.load('cities_40cities_40maps.npy')
lengths40 = np.load('lengths_40cities_40maps.npy')
params_opt = np.load('params_opt_40cities_40maps.npy',allow_pickle=True).all()

In [ ]:
orders40_simcim = np.load('orders40_simcim.npy',allow_pickle=True).all()
N_cities = 40
for key in ['manual']:
    print(key)
    orders40_simcim[key] = []
    for i in trange(N_cities):
        B = 0.1
        A = 1.5*B*lengths40[i].max()
        J,h = get_Jh(lengths40[i], A, B)
        J = J.to(device)
        h = h.to(device)
        simcim = Simcim(J, h, device, datatype)
        simcim.params_disc['attempt_num'] = 1500
        c_current, c_evol = simcim.evolve(simcim.params_cont)
        s_cur = torch.sign(c_current)
        E = energy(J,h,s_cur)
        s_min = s_cur[:,torch.argmin(E)]
        order_cur = get_order_simcim(s_min,N_cities).cpu().numpy()
        orders40_simcim[key].append(order_cur)

In [ ]:
# np.save('orders40_simcim',orders40_simcim)
orders40_simcim_nn = np.load('orders40_simcim_nn.npy',allow_pickle=True).all()
for key in orders40_simcim_nn.keys():
    orders40_simcim[key] = orders40_simcim_nn[key]

In [ ]:
lengths_simcim = {}
for key in orders40_simcim.keys():
    lengths_simcim[key] = []
    for i in range(40):
        lengths_simcim[key].append(length(torch.tensor(orders40_simcim[key][i]),torch.tensor(lengths40[i])).numpy())
    lengths_simcim[key] = np.array(lengths_simcim[key])
np.save('lengths_simcim',lengths_simcim)
lengths_simcim

In [ ]:
plt.plot(lengths_simcim['neural_net'],'.',label ='NN')
plt.plot(lengths_simcim['manual'],'.',label ='Manual')
plt.legend()

In [ ]:
lengths_opt = {}
keys = ['gaussian_process', 'neural_net', 'differential_evolution', 'random']
for key in keys:
    lengths_opt[key] = []
    for i in range(40):
        lengths_opt[key].append(length(torch.tensor(orders40[key][i]),torch.tensor(lengths40[i])).numpy())

In [ ]:
keys = ['gaussian_process', 'neural_net', 'differential_evolution', 'random']
for key in keys:
    plt.hist(lengths_opt[key],label = key,alpha = 0.8)
    plt.legend()
    plt.show()

In [ ]:
fig,ax = plt.subplots(40,2,figsize = (15,120))
for k in range(40):
        plot_cities(ax[k,0],cities40[k],orders40['nelder_mead'][k])
        ax[k,0].plot(np.zeros(5),'o',color='white',alpha=0.,label =# 'Длина маршрута '+
             str(np.around(length(torch.tensor(orders40['neural_net'][k]),torch.tensor(lengths40[k])).numpy(),2)))#+'\n алгоритм без нормировки')
        ax[k,0].legend(fontsize = 18,loc=1)
        
        plot_cities(ax[k,1],cities40[k],orders40['gaussian_process'][k])
        ax[k,1].plot(np.zeros(5),'o',color='white',alpha=0.,label =# 'Длина маршрута '+
             str(np.around(length(torch.tensor(orders40['gaussian_process'][k]),torch.tensor(lengths40[k])).numpy(),2)))#+'\n алгоритм без нормировки')
        ax[k,1].legend(fontsize = 18,loc=1)
        
plt.show()

In [ ]:
simcim.params_disc['attempt_num'] = 1000
c_current, c_evol = simcim.evolve(params_opt)

In [ ]:
s_cur = torch.sign(c_current)

fig, ax = plt.subplots(1,3,figsize=(15,5))
for i in range(c_evol.shape[0]):
    ax[0].plot(c_evol[i].cpu().numpy())
ax[0].set_xlabel('временной шаг', fontsize = 18)
ax[0].set_ylabel('амплитуда $c_i$', fontsize = 18)
ax[0].tick_params(labelsize=18)

E = energy(J,h,s_cur)
ax[1].hist(E.cpu().numpy(), label = '$H_{min} = $' + str(E.min().cpu().numpy()))
ax[1].set_xlabel('H',fontsize = 18)
ax[1].set_ylabel('Количество решений',fontsize = 18)
ax[1].tick_params(labelsize=18)
ax[1].legend(fontsize=18)

s_min = s_cur[:,torch.argmin(E)]
order = get_order_simcim(s_min,N_cities)

plot_cities(ax[2],cities,order.cpu().numpy())
ax[2].plot(np.zeros(5),'o',color='white',alpha=0.,label =# 'Длина маршрута '+
             str(np.around(length(order,torch.tensor(lengths)).numpy(),2)))#+'\n алгоритм без нормировки')
ax[2].legend(fontsize = 18,loc=1)
ax[2].tick_params(labelsize=18)

fig.tight_layout()
plt.show()

In [ ]:
params_opt